---
## Updated DDX56 Analysis for Pearson Lab
L.Richards   
September 2019  

---

1. Expression of DDX56 across all cells in G523
2. Co-expression of DDX56 and SOX2 in G523
3. Differential gene expression of DDX56+SOX2+ cells versus single negative or double negative in G523 (output list of DE genes in csv, and visualize in heatmap)
4. Mike hypothesized that double positive cells (DDX56+SOX2+) have nigher expression of EGFR – confirm this is true at single cell level

In [3]:
library(ggplot2)
library("Seurat", 
        lib ="/mnt/work1/users/pughlab/projects/BTSCs_scRNAseq/Manuscript/lib/" ) #v2.3.4
library(reshape2)
#library(vioplot)

setwd("~/Desktop/DDX56_August2019/")

In [4]:
### load in processed data for G523_L
load("~/pughlab/projects/BTSCs_scRNAseq/Manuscript_G607removed/CellCyle_Regression/scClustViz/opt_solution/G523_L_res.0.2.RData")

---
## 1.0 Plot expression of DDX56 across all cells in G523
---

'G523_DDX56_expression.pdf'

In [ ]:
pdf("G523_DDX56_expression.pdf")

##plot as a tSNE
FeaturePlot(object = BTSC, 
            features.plot = "DDX56", 
            cols.use = c("lightgrey", "blue"),
            reduction.use = "tsne",
            no.legend = FALSE
           )
##plot as UMAP
FeaturePlot(object = BTSC, 
            features.plot = "DDX56", 
            cols.use = c("lightgrey", "blue"),
            reduction.use = "umap",
            no.legend = FALSE
           )

##plot distribution
plot(sort(BTSC@data[grep("DDX56", rownames(BTSC@data)), ]),
     ylab = "Normalized DDX56 Expression",
     xlab = "Sorted Cells",
     main = "G523_L"
    
    )


dev.off()



## classify cells as DDX56+ and DDX56-
## any cell with DDX56 expression >0 classified as DDX56+

BTSC@meta.data$DDX56_class <- ifelse(BTSC@data[grep("DDX56", rownames(BTSC@data)), ] > 0,
                                     "DDX56+",
                                     "DDX56-"
                                    
                                    )
table(BTSC@meta.data$DDX56_class)
BTSC@ident <- factor(BTSC@meta.data$DDX56_class)
BTSC@ident[1:5]

## DDX56- = 2560 cells
## DDX56+ = 1185 cells

---
## 2.0 Co-expression of DDX56 and genes of interest
---

'G523_DDX56_coexpression.pdf'

Genes of interest:  
SOX2, IFNB1, TNFa, Il8, Rantes and Isg56

---
IFNB1 is not in the single cell data frame, other IF genes in the data are:  "IFFO2"   "IFI6"    "IFI44"   "IFI16"   "IFT172"  "IFRD2"   "IFT57"
 "IFT122"  "IFT80"   "IFNGR1"  "IFT22"   "IFRD1"   "IFT74"   "IFITM2" "IFITM3"  "IFT46"   "IFIT2"   "IFIT3"   "IFIT1"   "IFIT5"   "IFFO1" "IFT81"   "IFT88"   "IFT43"   "IFI27L1" "IFI27"   "IFI27L2" "IFT140" "IFT20"   "IFI35"   "IFT52"   "IFI30"   "IFT27"   "IFNAR2"  "IFNAR1" "IFNGR2"
 
 Not sure which TNFa gene is being referred to, these are the ones in the data: "TNFRSF1B"  "TNFAIP6"   "TNFRSF21"  "TNFAIP3"   "TNFRSF10B" "TNFRSF1A" "TNFRSF19"  "TNFRSF12A" "TNFSF12"   "TNFAIP1"   "TNFAIP8L1" "TNFSF9". 
 
 IL8 (aka CXCL8) is not detected in the data.
 
 RANTES (aka CCL5) is not detected in the data. 
 
 

In [ ]:
genes <- c("SOX2", 
          rownames(BTSC@data)[grep("^TNF", rownames(BTSC@data))], #include all the TNFs
           "EGFR",
           "IFIT1" #alternate name for ISG56
           )
           
           
### co-expression plots

pdf("G523_DDX56_coexpression.pdf")

for(i in 1:length(genes)){
    
#coexpression plor

a <- plot(BTSC@data["DDX56", ],
         BTSC@data[genes[i], ],
         xlab = "Normalized DDX56 Expression",
         ylab = paste0("Normalized ", genes[i], " Expression") ,
         main = "G523_L"
        )
print(a)
    
#compare expression between DDX56+/- cells

x1 <- BTSC@data[genes[i], BTSC@ident == "DDX56+"]
x2 <- BTSC@data[genes[i], BTSC@ident == "DDX56-"]

vioplot(x1, 
        x2,
        names=c("DDX56+", "DDX56"), 
        col=c("white", "grey"),
        ylab = "Normalized Expression"
       )
title(paste0("G523_L: ", genes[i]))
    
}

#plot all genes as heatmap

DoHeatmap(BTSC,
          genes.use = genes,
          group.by = "ident",
          remove.key = FALSE,
          cex.col = 0
         )


dev.off()

---
## 3.0 Compare EGFR expression between DDX56/SOX2 states
---

"G523_DDX56_SOX2_EGFR.pdf"

In [ ]:
## identify DDX56+/SOX2- cells

DDX56_SOX2 <- NULL
DDX56_SOX2[BTSC@data["DDX56", ] > 0 & BTSC@data["SOX2", ] > 0] <- "DDX56+SOX2+"
DDX56_SOX2[BTSC@data["DDX56", ] > 0 & BTSC@data["SOX2", ] == 0] <- "DDX56+SOX2-"    
DDX56_SOX2[BTSC@data["DDX56", ] == 0 & BTSC@data["SOX2", ] > 0] <- "DDX56-SOX2+"   
DDX56_SOX2[BTSC@data["DDX56", ] == 0 & BTSC@data["SOX2", ] == 0] <- "DDX56-SOX2-"   

BTSC@meta.data$DDX56_SOX2_class <- DDX56_SOX2

table(BTSC@meta.data$DDX56_SOX2_class)

BTSC@ident <- factor(BTSC@meta.data$DDX56_SOX2_class)
table(BTSC@ident)

BTSC <- SetIdent(BTSC, id = factor(BTSC@meta.data$DDX56_SOX2_class))

####DDX56-SOX2+ DDX56+SOX2- DDX56+SOX2+  DDX56-SOX2-
###      2533          16        1169        27    cells

## not plot EGFR between groups

x1 <- BTSC@data["EGFR", BTSC@ident == "DDX56+SOX2+"]
x2 <- BTSC@data["EGFR", BTSC@ident == "DDX56-SOX2+"]
x3 <- BTSC@data["EGFR", BTSC@ident == "DDX56+SOX2-"]
x4 <- BTSC@data["EGFR", BTSC@ident == "DDX56-SOX2-"]

pdf("G523_DDX56_SOX2_EGFR.pdf")

vioplot(x1, 
        x2,
        x3,
        x4,
        names=c("DDX56+SOX2+","DDX56-SOX2+", "DDX56+SOX2-", "DDX56-SOX2-"), 
        col=c("white"),
        ylab = "Normalized Expression"
       )
title(paste0("G523_L: ", "EGFR"))
 
dev.off()


---
## 4.0 Differential Gene Expression Analysis
---

"DE_top10genes_heatmap.pdf"


Differential gene expression of DDX56+SOX2+ cells versus single negative or double negative in G523 (output list of DE genes in csv, and visualize in heatmap).....There are not enough double negative cells to do this anlaysis (only 27)

Insead I will do DE analysis of each group versus all other cells. 

In [ ]:
### DDX56+SOX2+ versus all other cells

DE.genes <- FindAllMarkers(BTSC, 
                           min.pct = 0.5, #gene must be in at leat 50% of cells in wither group
                           only.pos = TRUE #only report positive markers for each group
                          )
write.csv(DE.genes, file = "DDX56_SOX2_DE_Genes.csv")

top10 <- DE.genes %>% group_by(cluster) %>% top_n(10, avg_logFC)

pdf("DE_top10genes_heatmap.pdf") 
#this look bad because of small numbers in some groups of cells

DoHeatmap(object = BTSC, 
          genes.use = top10$gene, 
          slim.col.label = TRUE, 
          remove.key = FALSE
         )

dev.off()

## note: most of the genes from the DE analysis are not significant when p vlaue is adjusted

